# Exploratory Data Analysis

---

1. Import packages
2. Loading data with Pandas
3. Descriptive statistics of data
4. Data visualization
5. Hypothesis investigation

---


## I. Import packages

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

---

## II. Loading data with Pandas

We need to load `Test_data.csv` and `pharma_data.csv` into individual dataframes so that we can work with them in Python

In [7]:
#Importing datasets
pharma_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_begs.csv")
Test_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_begs.csv")

Looking at the first 5 rows of both dataframes to see what the data looks like

In [11]:
pharma_data.head()

(23097, 18)

In [10]:
Test_data.head()

(9303, 17)

With the `pharma_data.csv`, we have a mix of numeric, categorical data and null values which we will need to transform before modelling.

---

## III. Descriptive statistics of data

### Data types

In [ ]:
pharma_data.info()

In [ ]:
pharma_data.describe()

The describe method gives us a lot of information about the data. The key point to take away from this is that we some  data, as exhibited by std

### Missing Values

In [ ]:
#Checking for null values
pharma_data.isna().sum()

Here, we can see that a handful of columns has null values present.

In [ ]:
# Filling null values with the next value on the column
pharma_data.Treated_with_drugs.fillna(method = "pad",inplace = True)
pharma_data.A.fillna(method = "pad",inplace = True)
pharma_data.B.fillna(method = "pad",inplace = True)
pharma_data.C.fillna(method = "pad",inplace = True)
pharma_data.D.fillna(method = "pad",inplace = True)
pharma_data.E.fillna(method = "pad",inplace = True)
pharma_data.F.fillna(method = "pad",inplace = True)
pharma_data.Z.fillna(method = "pad",inplace = True)

In [ ]:
# Summing all our columns and using those values as Number_of_prev
cols = pharma_data[["A","B","C","D","E","F","Z"]]
cols = cols.sum(axis = 1)
pharma_data.Number_of_prev_cond = cols

In [ ]:
#Now there are no null values
pharma_data.isna().sum()

---
## IV. Data visualization

Now let's create meaning and conclusions from the dataframes

### 1. Percentage of Survival

In [ ]:
survived = pharma_data[["ID_Patient_Care_Situation","Survived_1_year"]]
survived.columns = ["Patients","Survived"]
survived_total = survived.groupby(["Survived"]).count().sort_values(by = "Patients",ascending=False)
survived_percent = (survived_total/survived_total.sum())*100
survived_percent

In [ ]:
survived_percent.transpose().plot.bar(stacked = True,rot = 0)
plt.ylabel("Patient Percentage")
plt.legend(["Alive after 1Yr","Dead after 1Yr"],loc = "upper right")
plt.show()

About 36% of the total number are dead (8934 patients)


### 2. Patients who smoke

Let's see the distribution of the patients who smoke. Since the  data is uni-variate, let's use histograms to visualize their distribution.

In [ ]:
#There are rows that indicate "Cannot Say". Let's convert all the "Cannot Say" categories to YES OR NO
pharma_data.Patient_Smoker.replace("Cannot say","YES",inplace = True)
pharma_data

In [ ]:
smokers = pharma_data[["ID_Patient_Care_Situation","Patient_Smoker","Survived_1_year"]]
smokers.columns = ["id","Patient_Smoker","Survived"]
smoker_total = smokers.groupby([smokers["Patient_Smoker"],smokers["Survived"]])["id"].count().unstack()
smoker_percent =  (smoker_total.div(smoker_total.sum(axis = 1),axis = 0)*100).sort_values(by = 0,ascending = False)
smoker_percent

In [ ]:
smoker_percent.plot.bar(stacked = True,rot =0,figsize=(18,10))
plt.ylabel("Percentage %")
plt.xlabel("Patients who smoke")
plt.legend(["Dead","Alive"],loc = "upper center",fontsize =20)
plt.show()

Smoking doesn't affect the rate of a patient's survival that much
 - 49% of smokers died as 51% of the same smokers survived
 - 26% of non-smokers dies as 73% of the same non-smokers survived

### 3. RURAL or URBAN community

In [ ]:
Rural_urban = pharma_data[["ID_Patient_Care_Situation","Patient_Rural_Urban","Survived_1_year"]]
Rural_urban.columns = ["id","Patient_Rural_Urban","Survival"]
Rural_urban_total = Rural_urban.groupby([Rural_urban["Patient_Rural_Urban"],Rural_urban["Survival"]])["id"].count().unstack()
Rural_urban_percent = (Rural_urban_total.div(Rural_urban_total.sum(axis = 1),axis = 0)*100).sort_values(by = 0,ascending = False)
Rural_urban_percent

In [ ]:
Rural_urban_percent.plot.bar(stacked = True,rot = 0,figsize=(15,15))
plt.legend(["Dead","Alive"],loc = "upper center",fontsize =18)
plt.ylabel("Percentage %")
plt.xlabel("Part of Country")
plt.show()

Rural
- 66% of Rural Residents survived
- 33% of Urban Residents died

Urban
- 54% of Urban Residents survived
- 45% of Urban Residents died


### 4. Patients with previous category of condition

In [ ]:
Prev_con = pharma_data[["ID_Patient_Care_Situation","Number_of_prev_cond","Survived_1_year"]]
Prev_con.columns = ["id","Previous_con","Survival"]
Prev_con_total = Prev_con.groupby([Prev_con["Previous_con"],Prev_con["Survival"]])["id"].count().unstack()
Prev_con_percent = (Prev_con_total.div(Prev_con_total.sum(axis = 1),axis = 0)*100).sort_values(by = 0,ascending = True)
Prev_con_percent

In [ ]:
Prev_con_percent.plot.bar(stacked = True,rot = 0,figsize=(10,10))
plt.legend(["Dead","Alive"],loc = "lower right",fontsize = 15)
plt.ylabel("Percentage %")
plt.xlabel("Number of Previous Conditions")
plt.show()


---
This clearly shows the relationship between number of previous conditions and rate of survival
The more the number of previous conditions, the more the rate to death
- 94% of those with 5 previous conditions died
- 59% of those with 4 previous conditions died
- 41% of those with 3 previous conditions died
- 36% of those with 2 previous conditions died
- 34% of those with 1 previous condition died


### 5. Different Category of Previous Conditions

In [ ]:
cat_prev_condition = pharma_data[["ID_Patient_Care_Situation","A","B","C","D","E","F","Z"]]
cat_prev_condition = cat_prev_condition.replace(0,np.nan)
cat_prev_condition["Survival"] = pharma_data.Survived_1_year
cat_prev_condition

In [ ]:

a = cat_prev_condition.groupby([cat_prev_condition["A"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
b = cat_prev_condition.groupby([cat_prev_condition["B"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
c = cat_prev_condition.groupby([cat_prev_condition["C"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
d = cat_prev_condition.groupby([cat_prev_condition["D"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
e = cat_prev_condition.groupby([cat_prev_condition["E"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
f = cat_prev_condition.groupby([cat_prev_condition["F"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()
z = cat_prev_condition.groupby([cat_prev_condition["Z"],cat_prev_condition["Survival"]])["ID_Patient_Care_Situation"].count().unstack()

In [ ]:
a_percent = (a.div(a.sum(axis = 1),axis = 0)*100)
b_percent = (b.div(b.sum(axis = 1),axis = 0)*100)
c_percent = (c.div(c.sum(axis = 1),axis = 0)*100)
d_percent = (d.div(d.sum(axis = 1),axis = 0)*100)
e_percent = (e.div(e.sum(axis = 1),axis = 0)*100)
f_percent = (f.div(f.sum(axis = 1),axis = 0)*100)
z_percent = (z.div(z.sum(axis = 1),axis = 0)*100)

In [ ]:
a_percent.index = ["A"]
b_percent.index = ["B"]
c_percent.index = ["C"]
d_percent.index = ["D"]
e_percent.index = ["E"]
f_percent.index = ["F"]
z_percent.index = ["Z"]

In [ ]:
sets = [a_percent,b_percent,c_percent,d_percent,e_percent,f_percent,z_percent]
result = pd.concat(sets).sort_values(by = 1,ascending=False)
result

In [ ]:
result.plot.bar(stacked = True,rot = 0,figsize=(18,10))
plt.legend(["Dead","Alive"],loc = "upper right",fontsize =25)
plt.ylabel("Percentage %")
plt.xlabel("Patients with the Different Categories of Previous Diseases ")
plt.show()

---
This clearly shows the relationship between the rate of survival and the class of Previous condition they had
- 100% of those with condition Z survived
- 64% of those with condition E survived
- 63% of those with condition F survived

### 7.Treated with Drugs

In [ ]:
drugs = pharma_data[["ID_Patient_Care_Situation","Treated_with_drugs","Survived_1_year"]]
drugs.columns = ["id","drugs","Survival"]
drugs_total = drugs.groupby([drugs["drugs"],drugs["Survival"]])["id"].count().unstack()
drugs_percent = (drugs_total.div(drugs_total.sum(axis = 1),axis = 0)*100).sort_values(by = 1,ascending = False)
drugs_percent

In [ ]:
drugs_percent.plot.bar(stacked=True, rot="vertical", figsize=(20, 20))
plt.legend(["Dead", "Alive"], loc="upper center", fontsize=18)
plt.ylabel("Percentage %")
plt.xlabel("drugs")
plt.show()

Class of drugs used during treatment
- DX1 has 74% of Survivors
- DX2 has 71% of Survivors
- DX3 has 73% of Survivors

FOR DRUG COMBINATIONS the top 3 contributors of survivors are;

- DX1,DX2,DX3,DX4,DX5 has 100% survivors
- DX1,DX4,DX5 has 93% survivors
- DX2,DX5 has 88% survivors

### 8.Diagnosed Condition

In [ ]:
diagnosed = pharma_data[["ID_Patient_Care_Situation","Diagnosed_Condition","Survived_1_year"]]
diagnosed.columns = ["id","Diagnosed","Survival"]
new_df = pd.DataFrame({"Survived":diagnosed[diagnosed["Survival"]==1]["Diagnosed"],
                    "Died":diagnosed[diagnosed["Survival"]==0]["Diagnosed"]

                    })
new_df

In [ ]:
new_df[["Survived","Died"]].plot.hist(stacked = True)
plt.xlabel("Diagnosed Condition")
plt.show()

### 9. Patient Age

In [ ]:
age_pd = pharma_data[["ID_Patient_Care_Situation","Patient_Age","Survived_1_year"]]
age_pd.columns = ["id","Age","Survival"]
age = pd.DataFrame({"Survived":age_pd[age_pd["Survival"]==1]["Age"],
                     "Died":age_pd[age_pd["Survival"]==0]["Age"]

})
age

In [ ]:
age[["Survived","Died"]].plot.hist(stacked = True)
plt.xlabel("Age")
plt.show()

In [ ]:
#We will annotate to see which condition has the most survivors

### 10. Body Mass Ratio

In [ ]:
bmr = pharma_data[["ID_Patient_Care_Situation","Patient_Body_Mass_Index","Survived_1_year"]]
bmr.columns = ["id","Patient_Body_Mass_Index","Survival"]
new = pd.DataFrame({"Survived":bmr[bmr["Survival"]==1]["Patient_Body_Mass_Index"],
                     "Died":bmr[bmr["Survival"]==0]["Patient_Body_Mass_Index"]

                     })
new

In [ ]:
new[["Survived","Died"]].plot.hist(stacked = True)
plt.xlabel("Patient_Body_Mass_Index")
plt.show()

Clearly,`Patient_Body_Mass_Index` and `Patient_Age`data is highly negatively skewed, presenting a very long right-tail towards the higher values of the distribution. The values on the left are likely to be outliers. We can use a standard plot to visualise the outliers in more detail. A boxplot is a standardized way of displaying the distribution based on a five number summary:
- Minimum
- First quartile (Q1)
- Median
- Third quartile (Q3)
- Maximum

It can reveal outliers and what their values are. It can also tell us if our data is symmetrical, how tightly our data is grouped and if/how our data is skewed.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# Plotting a boxplot
fig, axs = plt.subplots(nrows=2, figsize=(20,15))
sns.boxplot(bmr["Patient_Body_Mass_Index"],ax=axs[0])
sns.boxplot(age_pd["Age"],ax=axs[1])
plt.plot()

Looks like there are outliers and skewness, and we will deal with them in the feature engineering

---
## V. FEATURE ENGINEERING
#### Transforming Boolean Data

In [ ]:
pharma_data["Patient_Smoker"] = pharma_data.Patient_Smoker.replace(["YES","NO"],[1,0])

In [ ]:
pharma_data.groupby(["Patient_Smoker"]).agg({"Survived_1_year":"mean"})

It is obvious that being a patient smoker affects their tendency to survive.

#### Transforming numerical data
In EDA some variables were highly skewed. The reason why we need to treat skewness is because some predictive models have inherent assumptions about the distribution of the features that are being supplied to it. Such models are called parametric models, and they typically assume that all variables are both independent and normally distributed.

Skewness isn’t always a bad thing, but as a rule of thumb it is always good practice to treat highly skewed variables because of the reason stated above, but also as it can improve the speed at which predictive models are able to converge to its best solution.

There are many ways that you can treat skewed variables. You can apply transformations such as:
Square root
Cubic root
Logarithm
 For this use case we will use the ‘Logarithm’ transformation for the positively skewed features.
Note: We cannot apply log to a value of 0, so we will add a constant of 1 to all the values
First I want to see the statistics of the skewed features, so that we can compare before and after transformation

In [ ]:
pharma_data[["Patient_Age","Patient_Body_Mass_Index","Diagnosed_Condition"]].describe()

In [ ]:
pharma_data["Patient_Age"] = np.log10(pharma_data["Patient_Age"] + 1)
pharma_data["Patient_Body_Mass_Index"] = np.log10(pharma_data["Patient_Body_Mass_Index"] + 1)
pharma_data["Diagnosed_Condition"] = np.log10(pharma_data["Diagnosed_Condition"] + 1)
#Checking the skewness now
pharma_data[["Patient_Age","Patient_Body_Mass_Index","Diagnosed_Condition"]].describe()

In [ ]:
fig, axs = plt.subplots(nrows=3,figsize=(18,18))
# Plot histograms
sns.distplot((pharma_data["Patient_Age"].dropna()), ax=axs[0])
sns.distplot((pharma_data["Patient_Body_Mass_Index"].dropna()), ax=axs[1])
sns.distplot((pharma_data["Diagnosed_Condition"].dropna()), ax=axs[2])
plt.show()

#### TRANSFORMING CATEGORICAL DATA

A predictive model cannot accept categorical or string values.
The simplest method is to map each category to an integer (label encoding/Ordinal encoding), however this is not always appropriate beecause it then introduces the concept of an order into a feature which may not inherently be present 0 < 1 < 2 < 3 ...
Another way to encode categorical features is to use dummy variables AKA one hot encoding. This create a new feature for every unique value of a categorical column, and fills this column with either a 1 or a 0 to indicate that this company does or does not belong to this category.

In [ ]:
pharma_data.Patient_Rural_Urban = pharma_data.Patient_Rural_Urban.astype("category")
pharma_data.Treated_with_drugs = pharma_data.Treated_with_drugs.astype("category")
pharma_data.Number_of_prev_cond = pharma_data.Number_of_prev_cond.astype("category")
pharma_data.Diagnosed_Condition = pharma_data.Diagnosed_Condition.astype("category")
pharma_data.Patient_Age = pharma_data.Patient_Age.astype("category")
pharma_data

In [ ]:
pharma_data = pd.get_dummies(pharma_data,columns=["Patient_Rural_Urban",
                                                  "Treated_with_drugs",
                                                  "Number_of_prev_cond",
                                                  "Diagnosed_Condition",
                                                  "Patient_Age"


])
pharma_data

We have 2 categories, so we will create 2 dummy variables from this column

Now we can see that for the majority of the features, their standard deviation is much lower after transformation. This is a good thing, it shows that these features are more stable and predictable now. Let’s quickly check the distributions of some of these features too.

# MODEL


We now have a dataset containing features that we have engineered and we are ready to start training a predictive model. We are only focused on training a Random Forest classifier.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

### Data sampling
The first thing we want to do is split our dataset into training and test samples. The reason why we do this, is so that we can simulate a real life situation by generating predictions for our test sample, without showing the predictive model these data points. This gives us the ability to see how well our model is able to generalise to new data, which is critical.

In [14]:
target = pharma_data["Survived_1_year"]
feature = pharma_data.drop(columns = ["Patient_ID",
                                      "ID_Patient_Care_Situation",
                                      "Survived_1_year",
                                      "Patient_mental_condition"
])

In [15]:
print(target.shape)
print(feature.shape)

In [16]:
#Divide feature and target into train and test
feature_train,feature_test,target_train,target_test = train_test_split(feature,target,test_size=25,random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [9303, 23097]

In [ ]:
print(target_test.shape)
print(feature_test.shape)
print(target_train.shape)
print(feature_train.shape)

We are using a Random Forest classifier in this example. A Random Forest sits within the category of ensemble algorithms because internally the Forest refers to a collection of Decision Trees which are tree-based learning algorithms.

In [ ]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(feature_train,target_train)

### Evaluation
Now let’s evaluate how well this trained model is able to predict the values of the test dataset. We are going to use 3 metrics to evaluate performance:
- Accuracy = the ratio of correctly predicted observations to the total observations
- Precision = the ability of the classifier to not label a negative sample as positive
- Recall = the ability of the classifier to find all the positive samples
- F1 Score

In [ ]:
prediction = model.predict(feature_test)
tn,fp,fn,tp = metrics.confusion_matrix(target_test,prediction).ravel()

In [ ]:
print(f"True Negative: {tn}")
print(f"True Positive: {tp}")
print(f"False Negative: {fn}")
print(f"False Positive: {fp}")

In [ ]:
print(f"Precision: {metrics.precision_score(target_test,prediction)*100}%")
print(f"Accuracy: {metrics.accuracy_score(target_test,prediction)*100}%")
print(f"Recall: {metrics.recall_score(target_test,prediction)*100}%")
print(f"F1 Score: {metrics.f1_score(target_test,prediction)*100}%")

In [ ]:
feature_importances = pd.DataFrame({"feature":feature_train.columns,
                                    "importance":model.feature_importances_


}).sort_values(by = "importance",ascending=False).reset_index()
feature_importances

In [ ]:
plt.figure(figsize=(20, 20))
plt.title('Feature Importances')
plt.barh(range(len(feature_importances)), feature_importances['importance'], color='b')
plt.yticks(range(len(feature_importances)), feature_importances['feature'])
plt.xlabel('Importance')
plt.show()

In [ ]:
target_test = target_test.reset_index()
target_test.drop(columns='index', inplace=True)
target_test.to_csv('prediction.csv')


In [ ]:
target_test["Survival"] = prediction.tolist()
# target_test.to_csv('prediction.csv')
target_test